## Transorming circuits data and storing it in silver layer for future transformations

####  using run commands to access notebook to run codes to configuring the adlsgen2 and connect

In [0]:
%run "../includes/adlsconfig"

In [0]:
%run "../includes/containersconfig"

In [0]:
#importing all the required functions
from pyspark.sql.types import *
from pyspark.sql.functions import *

### Defining the Schema for the circuits csv to read the file 

In [0]:
circuit_struct_schema =  StructType([
     StructField("circuitId", IntegerType(), False),
     StructField("circuitRef", StringType(), True),
     StructField("name", StringType(), True),
     StructField("location", StringType(), True),
     StructField("country", StringType(), True),
     StructField("lat", DoubleType(), True),
     StructField("lng", DoubleType(), True),
     StructField("alt", IntegerType(), True),
     StructField("url", StringType(), True)
])

In [0]:
# reading the circuits file 
circuits_df = spark.read.format('csv').option('header', True).schema(circuit_struct_schema)\
    .load(f"{raw_folder_path}/circuits.csv")

In [0]:
circuits_df.display()

In [0]:
#selecting the required columns instead of all
df_selected_circuits = circuits_df.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))

In [0]:
#changing column names to something more defined using withColumnsRenamed
df_renamed_curcuits = df_selected_circuits.withColumnsRenamed({"circuitId" : "circuit_id",
                                         "circuitRef" : "circuit_ref",
                                         "lat" : "latitude",
                                         "lng" : "longitude",
                                         "alt" : "altitude" })

In [0]:
#adding a column (ingested date) to enter and display the ingested date and time to track the data
df_final_curcuits = df_renamed_curcuits.withColumn("ingested_date", current_timestamp())

In [0]:
df_final_curcuits.display()

In [0]:
# writing the data into silver folder using overwrite method in parquet format
df_final_curcuits.write.format('parquet').mode("overwrite").option("path",f"{silver_folder_path}/circuits").save()

In [0]:
# checking whether the written data is correct or needs to be updated
df = spark.read.format('parquet').option('inferSchema', True).option('header', True).load(f"{silver_folder_path}/circuits/")

In [0]:
df.display()